In [1]:
import pyodbc
print(pyodbc.drivers())
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=157.119.230.120,4071;"
    "Database=PowerBI_LISV2;"
    "UID=sa;"
    "PWD=Y0m@Sql.!.123;"
)

['SQL Server', 'ODBC Driver 17 for SQL Server', 'ODBC Driver 18 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [2]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(conn_str)

In [6]:
# Query
query = """SELECT distinct State, City,[LISStoreCode],[StoreName],[ChainName],[InLatitude],[InLongitude]
FROM [PowerBI_LISV2].[PidiliteManagement].[ISPStoreAttendance_NormalExport]
WHERE ([LISStoreCode] NOT LIKE '%Test%' OR [LISStoreCode] NOT LIKE '%test%');"""
df = pd.read_sql(query, conn)

C:\Users\kunal\AppData\Local\Temp\ipykernel_10308\94442313.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [7]:
df.head()

State   City LISStoreCode             StoreName ChainName  InLatitude  \
0  Bihar  Patna  Pidilite-48  BB-PATNA-PNM MALL-LO  Reliance   25.633846   
1  Bihar  Patna  Pidilite-48  BB-PATNA-PNM MALL-LO  Reliance  25.6338512   
2  Bihar  Patna  Pidilite-48  BB-PATNA-PNM MALL-LO  Reliance  25.6338539   
3  Bihar  Patna  Pidilite-48  BB-PATNA-PNM MALL-LO  Reliance  25.6338635   
4  Bihar  Patna  Pidilite-48  BB-PATNA-PNM MALL-LO  Reliance  25.6338696   

  InLongitude  
0  85.1065994  
1  85.1061755  
2  85.1061804  
3  85.1061658  
4   85.106163

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5226 entries, 0 to 5225
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   State         5226 non-null   object
 1   City          5226 non-null   object
 2   LISStoreCode  5226 non-null   object
 3   StoreName     5226 non-null   object
 4   ChainName     5226 non-null   object
 5   InLatitude    5226 non-null   object
 6   InLongitude   5226 non-null   object
dtypes: object(7)
memory usage: 285.9+ KB


In [9]:
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

from sklearn.preprocessing import StandardScaler
from math import radians

In [14]:
# !pip install shapely

In [10]:
df['latitude'] = df['InLatitude'].astype(float)
df['longitude'] = df['InLongitude'].astype(float)

In [11]:

# To store results
store_centroids = []

# Group by store and process each separately
for LISStoreCode, group in df.groupby('LISStoreCode'):
    coords = group[['latitude', 'longitude']].to_numpy()
    coords_rad = np.radians(coords)

    # DBSCAN parameters
    kms_per_radian = 6371.0088
    epsilon = 0.8 / kms_per_radian  # 800 meters
    db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(coords_rad)

    group = group.copy()
    group['cluster'] = db.labels_

    # Filter out noise
    clustered = group[group['cluster'] != -1]

    if clustered.empty:
        # print(f"No valid cluster for store {LISStoreCode}")
        continue

    # Pick largest cluster
    main_cluster = clustered['cluster'].value_counts().idxmax()
    main_cluster_points = clustered[clustered['cluster'] == main_cluster]

    # Compute centroid of the main cluster
    centroid_lat = main_cluster_points['latitude'].mean()
    centroid_lon = main_cluster_points['longitude'].mean()

    store_centroids.append({
        'store_id': LISStoreCode,
        'centroid_latitude': centroid_lat,
        'centroid_longitude': centroid_lon
    })



In [12]:
# Final result
centroid_df = pd.DataFrame(store_centroids)

In [13]:
centroid_df.head(10)

store_id  centroid_latitude  centroid_longitude
0  Pidilite-002          12.893461           77.567248
1  Pidilite-003          13.014585           77.554620
2  Pidilite-004          28.660619           77.297043
3  Pidilite-005          31.670284           74.882057
4  Pidilite-006          19.218697           72.865959
5  Pidilite-007          12.993597           77.700411
6  Pidilite-008          26.882858           81.056734
7  Pidilite-009          26.799187           81.003911
8  Pidilite-010          26.784547           80.992913
9  Pidilite-011          10.028159           76.307811

In [14]:
centroid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   store_id            77 non-null     object 
 1   centroid_latitude   77 non-null     float64
 2   centroid_longitude  77 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.9+ KB


In [15]:
centroid_df.to_csv('centroid_df_pidi.csv')

In [49]:
# coords = df[['latitude', 'longitude']].to_numpy()

In [20]:
# coords_rad = np.radians(coords)

In [22]:
# kms_per_radian = 6371.0088
# epsilon = 0.1 / kms_per_radian  # 100 meters

In [23]:
# db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(coords_rad)

In [24]:
labels = db.labels_
# df['cluster'] = labels

In [50]:
# group = group.copy()
# group['cluster'] = db.labels_